In [1]:
!pip install -q tensorflow-addons==0.9.1
!pip install -q image-classifiers==1.0.0
!pip install -q efficientnet==1.1.0

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import math, re, os, glob, time, random
from collections import namedtuple
from functools import partial

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

from classification_models.tfkeras import Classifiers
import efficientnet.tfkeras as efn

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from kaggle_datasets import KaggleDatasets

import seaborn as sns
from matplotlib import pyplot as plt

print("Tensorflow version " + tf.__version__)

K = tf.keras.backend
L = tf.keras.layers
AUTO = tf.data.experimental.AUTOTUNE

/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.1.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


Tensorflow version 2.3.0


# TPU/GPU detection

In [3]:
# Detect hardware, return appropriate distribution strategy
def get_strategy():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

    print("REPLICAS: ", strategy.num_replicas_in_sync)
    
    return strategy

strategy = get_strategy()

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


# Configuration

In [4]:
# data setup
SEED = 601
GCS_PATH = KaggleDatasets().get_gcs_path('shopee-product-detection-open')  # used in tpu kernel
GCS_PATH_2 = KaggleDatasets().get_gcs_path('sp-tfrecords-sz320')  # 'sp-tfrecords' or 'sp-tfrecords-sz320'
DATA_PATH = '../input/shopee-product-detection-open/'

MODEL_PATH = '../input/sp-rn50/'
MODEL_PATH_2 = '../input/sp-rn50-cycs/'
MODEL_PATH_3 = '../input/sp-rn50-folds/'

MODEL_PATH_1_1 = '../input/sp-effnb4/'
MODEL_PATH_1_2 = '../input/sp-effnb4-cycs/'
MODEL_PATH_1_3 = '../input/sp-effnb4-folds/'

MODEL_PATH_2_1 = '../input/sp-effnb5/'
MODEL_PATH_2_2 = '../input/sp-effnb5-cycs/'
MODEL_PATH_2_3 = '../input/sp-effnb5-folds/'
MODEL_PATH_2_4 = '../input/sp-effnb5-folds-2/'

CLASSES = sorted(os.listdir(DATA_PATH+'train/train/train'))
IM_SZ = 320  # 224, 320, 384, 448, 512
IMAGE_SIZE = [IM_SZ, IM_SZ]
WHICH_FOLD = 0

# training setup
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

random.seed(SEED)
np.random.seed(SEED)

## Functions

In [5]:
# numpy and matplotlib defaults
np.set_printoptions(threshold=15, linewidth=80)

def batch_to_numpy_images_and_labels(data, to_deprocess=False):
    try:
        images, labels = data
        numpy_images = images.numpy()
        numpy_labels = labels.numpy()
    except:
        images = data
        numpy_images = images.numpy()
        numpy_labels = None
    # If no labels, only image IDs, return None for labels (this is the case for test data)
    if to_deprocess:
        numpy_images = np.stack([deprocessing(im) for im in numpy_images])
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_sample(image, title, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)
    
def display_batch_of_images(databatch, predictions=None, to_deprocess=False):
    """This will work with:
    display_batch_of_images(images)
    display_batch_of_images(images, predictions)
    display_batch_of_images((images, labels))
    display_batch_of_images((images, labels), predictions)
    """
    # data
    images, labels = batch_to_numpy_images_and_labels(databatch, to_deprocess)
    if labels is None:
        labels = [None for _ in enumerate(images)]
    else:
        labels = np.argmax(labels, -1)
        
    # auto-squaring: this will drop data that does not fit into square or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
        
    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols,1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
    
    # display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else CLASSES[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target(predictions[i], label)
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one_sample(image, title, subplot, not correct, titlesize=dynamic_titlesize)
    
    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()

def display_confusion_matrix(cmat, score, precision, recall):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(CLASSES)))
    ax.set_xticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(CLASSES)))
    ax.set_yticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()

def display_training_curves(training, validation, title, subplot, figsize=(8,10)):
    if subplot % 10 == 1:  # set up the subplots on the first call
        plt.subplots(figsize=figsize)
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

# Datasets

In [6]:
train_df = pd.read_csv(DATA_PATH + 'train.csv')
test_df = pd.read_csv(DATA_PATH + 'test.csv')

NUM_TRAIN_IMAGES = train_df.shape[0]
NUM_TEST_IMAGES = test_df.shape[0]

print('Num. of train images:', NUM_TRAIN_IMAGES)
print('Num. of test images:', NUM_TEST_IMAGES)

Num. of train images: 105390
Num. of test images: 12186


In [7]:
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH_2 + '/train/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH_2 + '/test/*.tfrec')

## Data pipeline

In [8]:
# for caffe preproc mode
MEAN_C = [123.68, 116.779, 103.939]

# for torch preproc mode
MEAN_T = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

def decode_image(image, mode='torch'):
    assert mode in (None, 'tf', 'torch', 'caffe'), "mode must be one of None, 'tf', 'torch', 'caffe'"
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, IMAGE_SIZE)
    if mode is None:
        return image
    if mode == 'tf':
        image = image / 255.0  # convert image to floats in [0, 1] range
    if mode == 'torch':
        image = image / 255.0
        image = (image - MEAN_T) / STD
    if mode == 'caffe':
        image = image - MEAN_C
    return image

def deprocessing(image, mode='torch'):
    assert mode in (None, 'tf', 'torch', 'caffe'), "mode must be one of None, 'tf', 'torch', 'caffe'"
    def rescale(x):
        low, high = x.min(), x.max()
        x_rescaled = (x - low) / (high - low)
        return x_rescaled
    if mode is None or mode == 'tf':
        return rescale(image)
    if mode == 'torch':
        return rescale(image * STD + MEAN_T)
    if mode == 'caffe':
        return rescale(image + MEAN_C)

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "label": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.one_hot(tf.cast(example['label'], tf.int32), len(CLASSES))
    return image, label

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),  # tf.string means bytestring
        "filename": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    filename = example['filename']
    return image, filename

def data_augment(image, label, p_hsv=0.6, p_affine=0.75, p_cutout=0.):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    r_hsv = tf.random.uniform([1], minval=0, maxval=1, dtype='float32', seed=SEED)
    r_affine = tf.random.uniform([1], minval=0, maxval=1, dtype='float32', seed=SEED)
    r_cutout = tf.random.uniform([1], minval=0, maxval=1, dtype='float32', seed=SEED)
    
    image = tf.image.random_flip_left_right(image)
    if r_hsv < p_hsv:
        image = tf.image.random_brightness(image, 0.15)
        image = tf.image.random_saturation(image, 0.8, 2.3)
        image = tf.image.random_contrast(image, 0.8, 1.3)
    if r_affine < p_affine:
        image = shift_scale_rotate(image, h_shift=0.1*IM_SZ, w_shift=0.1*IM_SZ)
    if r_cutout < p_cutout:
        image = cutout(image)
    
    return image, label

def get_training_dataset():
    dataset = tf.data.TFRecordDataset(TRAIN_FILENAMES, num_parallel_reads=AUTO)
    return (
        dataset
        .map(read_labeled_tfrecord, num_parallel_calls=AUTO)
        .repeat()
        .shuffle(100000)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )

def get_validation_dataset():
    dataset = tf.data.TFRecordDataset(VALID_FILENAMES, num_parallel_reads=AUTO)
    return (
        dataset
        .map(read_labeled_tfrecord, num_parallel_calls=AUTO)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )

def get_validation_dataset_tta():
    dataset = tf.data.TFRecordDataset(VALID_FILENAMES, num_parallel_reads=AUTO)
    return (
        dataset
        .map(read_labeled_tfrecord, num_parallel_calls=AUTO)
        .map(data_augment, num_parallel_calls=AUTO)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )

def get_test_dataset():
    dataset = tf.data.TFRecordDataset(TEST_FILENAMES, num_parallel_reads=AUTO)
    return (
        dataset
        .map(read_unlabeled_tfrecord, num_parallel_calls=AUTO)
        .batch(BATCH_SIZE)
    )

def get_test_dataset_tta():
    dataset = tf.data.TFRecordDataset(TEST_FILENAMES, num_parallel_reads=AUTO)
    return (
        dataset
        .map(read_unlabeled_tfrecord, num_parallel_calls=AUTO)
        .map(data_augment, num_parallel_calls=AUTO)
        .batch(BATCH_SIZE)
    )

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

## Data augmentation

In [9]:
def transform(image, matrix, border_mode=0):
    BORDERS = ['constant', 'nearest']
    border_mode = BORDERS[border_mode]
    
    DIM = IMAGE_SIZE[0]
    C = DIM // 2
    
    # LIST DESTINATION PIXEL INDICES
    y, x = tf.meshgrid(tf.range(DIM), tf.range(DIM))
    x_c, y_c = tf.reshape(x - C, [-1]), tf.reshape(y - C, [-1])
    x, y = tf.reshape(x, [-1]), tf.reshape(y, [-1])
    if matrix.shape[0] == 2:
        coord = tf.stack( [x_c,y_c] )   # (2, DIM*DIM)
    else:
        z = tf.ones([DIM*DIM], dtype='int32')
        coord = tf.stack( [x_c,y_c,z] )   # (3, DIM*DIM)
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    coord_pr = tf.matmul(matrix, tf.cast(coord, dtype='float32'))
    coord_pr = tf.cast(tf.round(coord_pr[:2,:] + C), dtype='int32')   # (2, DIM*DIM)
    
    # FIND ORIGIN PIXEL VALUES
    if border_mode == 'constant':
        x_pr, y_pr = coord_pr[0,:], coord_pr[1,:]
        outside_ind = tf.logical_or( tf.logical_or(y_pr>DIM-1 , y_pr<0), tf.logical_or(x_pr>DIM-1 , x_pr<0))

        x_pr = tf.boolean_mask(x_pr, tf.logical_not(outside_ind))   # (<DIM*DIM, )
        y_pr = tf.boolean_mask(y_pr, tf.logical_not(outside_ind))   # (<DIM*DIM, )
        x    = tf.boolean_mask(x, tf.logical_not(outside_ind))   # (<DIM*DIM, )
        y    = tf.boolean_mask(y, tf.logical_not(outside_ind))   # (<DIM*DIM, )

        coord_pr = tf.transpose( tf.stack( [x_pr, y_pr] ) )   # (<DIM*DIM, 2)
        coord = tf.cast(tf.transpose( tf.stack( [x, y] ) ), 'int64')   # (<DIM*DIM, 2)

        im_channels = tf.split(image, 3, axis=-1)
        rot_channels = []
        for im_val in im_channels:
            rot_val = tf.squeeze(tf.gather_nd(im_val, coord_pr), axis=-1)   # (<DIM*DIM, )
            rot = tf.SparseTensor(coord, rot_val, [DIM, DIM])
            rot_channels.append(tf.sparse.to_dense(rot, default_value=0, validate_indices=False))

        rot_image = tf.transpose(tf.stack(rot_channels), [1, 2, 0])   # (DIM, DIM, 3)
    
    if border_mode == 'nearest':
        coord_pr = tf.clip_by_value(coord_pr, 0, DIM - 1)   # (2, DIM*DIM)
        rot_image = tf.reshape( tf.gather_nd(image, tf.transpose(coord_pr)), [DIM, DIM, 3] )
    
    return rot_image

def rotate(image, angle):
    angle = math.pi * angle / 180.
    c1 = tf.math.cos(angle)
    s1 = tf.math.sin(angle)
    t_matrix = tf.reshape( tf.stack([c1,s1, -s1,c1]), [2,2] )
    return transform(image, t_matrix)

def shear(image, angle):
    angle = math.pi * angle / 180.
    c1 = tf.math.cos(angle)
    s1 = tf.math.sin(angle)
    one = tf.constant([1], dtype='float32')
    zero = tf.constant([0], dtype='float32')
    t_matrix = tf.reshape( tf.stack([one[0],s1, zero[0],c1]), [2,2] )
    return transform(image, t_matrix)

def zoom(image, height_zoom, width_zoom):
    one = tf.constant([1], dtype='float32')
    zero = tf.constant([0], dtype='float32')
    t_matrix = tf.reshape( tf.stack([one[0]/height_zoom,zero[0], zero[0],one[0]/width_zoom]), [2,2] )
    return transform(image, t_matrix)

def shift(image, height_shift, width_shift):
    one = tf.constant([1], dtype='float32')
    zero = tf.constant([0], dtype='float32')
    t_matrix = tf.reshape( tf.stack([one[0],zero[0],height_shift, zero[0],one[0],width_shift, zero[0],zero[0],one[0]]), [3,3] )
    return transform(image, t_matrix)

def shift_scale_rotate(image, rotation=15, shear=8, h_zoom=1, w_zoom=1, h_shift=20, w_shift=20):
    rot = rotation * tf.random.normal([1], dtype='float32')
    shr = shear * tf.random.normal([1], dtype='float32')
    h_zoom = h_zoom + tf.random.normal([1], dtype='float32') / 10.
    w_zoom = w_zoom + tf.random.normal([1], dtype='float32') / 10.
    h_shift = h_shift * tf.random.normal([1], dtype='float32')
    w_shift = w_shift * tf.random.normal([1], dtype='float32')
    
    angle = math.pi * rot / 180.
    c1 = tf.math.cos(angle)
    s1 = tf.math.sin(angle)
    one = tf.constant([1], dtype='float32')
    zero = tf.constant([0], dtype='float32')
    rot_matrix = tf.reshape( tf.concat([c1,s1, -s1,c1], axis=0), [2,2] )
    
    angle = math.pi * shr / 180.
    c1 = tf.math.cos(angle)
    s1 = tf.math.sin(angle)
    shr_matrix = tf.reshape( tf.concat([one,s1, zero,c1], axis=0), [2,2] )
    
    z_matrix = tf.reshape( tf.concat([one/h_zoom,zero, zero,one/w_zoom], axis=0), [2,2] )
    
    t_matrix = K.dot(K.dot(rot_matrix, shr_matrix), z_matrix)
    t_image = transform(image, t_matrix)
    
    s_matrix = tf.reshape( tf.concat([one,zero,h_shift, zero,one,w_shift, zero,zero,one], axis=0), [3,3] )
    return transform(t_image, s_matrix)

def cutout(image, min_height=0.4, min_width=0.4, max_height=0.6, max_width=0.6):
    DIM = IMAGE_SIZE[0]
    
    cut_height = tf.cast(tf.round(tf.random.uniform([], minval=min_height*DIM, maxval=max_height*DIM)), 'int32')
    cut_width = tf.cast(tf.round(tf.random.uniform([], minval=min_width*DIM, maxval=max_width*DIM)), 'int32')
    x_min = tf.random.uniform([], minval=-cut_width//2, maxval=DIM-1-cut_width//2, dtype='int32')
    x_max = x_min + cut_width
    y_min = tf.random.uniform([], minval=-cut_height//2, maxval=DIM-1-cut_height//2, dtype='int32')
    y_max = y_min + cut_height
    if x_min < 0:
        cut_width -= 0 - x_min
        x_min = tf.clip_by_value(x_min, 0, x_max)
    if y_min < 0:
        cut_height -= 0 - y_min
        y_min = tf.clip_by_value(y_min, 0, y_max)
    if x_max > DIM:
        cut_width -= x_max - DIM
        x_max = tf.clip_by_value(x_max, x_min, DIM)
    if y_max > DIM:
        cut_height -= y_max - DIM
        y_max = tf.clip_by_value(y_max, y_min, DIM)
    
    cut_area = tf.zeros([cut_height, cut_width, 3], dtype='float32')
    pad_top = y_min
    pad_bottom = DIM - y_max
    pad_left = x_min
    pad_right = DIM - x_max
    cut_mask = tf.pad(cut_area, [[pad_top, pad_bottom], [pad_left, pad_right], [0,0]], constant_values=1)
    
    cut_image = tf.multiply(image, cut_mask)
    return cut_image

## Train/validation split

In [10]:
VALID_FILENAMES = [TRAINING_FILENAMES[WHICH_FOLD]]
TRAIN_FILENAMES = TRAINING_FILENAMES[:WHICH_FOLD] + TRAINING_FILENAMES[WHICH_FOLD+1:]

In [11]:
NUM_TRAIN_IMAGES = count_data_items(TRAIN_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALID_FILENAMES)
STEPS_PER_EPOCH = math.ceil(NUM_TRAIN_IMAGES / BATCH_SIZE)
# VALIDATION_STEPS = math.ceil(NUM_VALIDATION_IMAGES / BATCH_SIZE)
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAIN_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

Dataset: 100121 training images, 5269 validation images, 12186 unlabeled test images


## Dataset visualizations

In [12]:
# val_dataset = get_validation_dataset()
# val_dataset = val_dataset.unbatch().batch(5)
# val_batch = iter(val_dataset)
# images, labels = next(val_batch)

In [13]:
# plt.figure(figsize=(5,5))
# display_one_sample(images[1], '', (1,1,1));

In [14]:
# plt.figure(figsize=(5,5))
# display_one_sample(shift_scale_rotate(images[1]), '', (1,1,1));

In [15]:
# plt.figure(figsize=(5,5))
# display_one_sample(cutout(images[1]), '', (1,1,1));

In [16]:
# # Peek at training data
# train_dataset = get_training_dataset()
# train_dataset = train_dataset.unbatch().batch(20)
# train_batch = iter(train_dataset)

In [17]:
# # run this cell again for next set of images
# # longer because of the augmentation process
# display_batch_of_images(next(train_batch), to_deprocess=True)

In [18]:
# val_dataset = get_validation_dataset()
# val_dataset = val_dataset.unbatch().batch(20)
# val_batch = iter(val_dataset)

In [19]:
# # run this cell again for next set of images
# display_batch_of_images(next(val_batch), to_deprocess=True)

In [20]:
# # peer at test data
# test_dataset = get_test_dataset()
# test_dataset = test_dataset.unbatch().batch(20)
# test_batch = iter(test_dataset)

In [21]:
# # run this cell again for next set of images
# display_batch_of_images(next(test_batch), to_deprocess=True)

# Keras

## Model

In [22]:
class ConcatPooling2D(L.Layer):
    "Layer that concats `GlobalAveragePooling2D` and `GlobalMaxPooling2D`,"
    def __init__(self):
        "Output will be 2*output_size or 2 if output_size is None"
        super().__init__()
        self.ap = L.GlobalAveragePooling2D()
        self.mp = L.GlobalMaxPooling2D()
    def call(self, x): return tf.concat([self.mp(x), self.ap(x)], 1)

In [23]:
strategy = get_strategy()

with strategy.scope():
    pretrained_model_1 = efn.EfficientNetB4(weights='noisy-student', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    pretrained_model_2 = efn.EfficientNetB5(weights='noisy-student', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
#     resnet, _ = Classifiers.get('resnet50')
#     pretrained_model = resnet(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
#     pretrained_model.trainable = True  # False = transfer learning, True = fine-tuning
    
    model_1 = tf.keras.Sequential([
        pretrained_model_1,
        L.GlobalAveragePooling2D(),
        L.Dense(len(CLASSES), activation='softmax')
    ])
    
    model_2 = tf.keras.Sequential([
        pretrained_model_2,
        L.GlobalAveragePooling2D(),
        L.Dense(len(CLASSES), activation='softmax')
    ])

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8
115261440/115255328 [==============================] - 4s 0us/step


# Validation

In [24]:
def TTA(num_tta, model, beta=0.4, mode='test'):
    assert mode in ('valid', 'test')
    original_pred = 0
    transformed_pred = 0
    for i in range(num_tta):
        if i == 0:
            dataset = get_validation_dataset() if mode=='valid' else get_test_dataset()
            images_ds = dataset.map(lambda image, _: image)
            original_pred += model.predict(images_ds)
        else:
            dataset = get_validation_dataset_tta() if mode=='valid' else get_test_dataset_tta()
            images_ds = dataset.map(lambda image, _: image)
            transformed_pred += model.predict(images_ds) / (num_tta-1)

    predictions = beta * original_pred + (1-beta) * transformed_pred
    return predictions

class NNAverage(object):
    def __init__(self, base_model, mu):
        self.mu = mu
        self.weight_copy = {}
        for var in base_model.trainable_variables:
            self.weight_copy[var.name] = 0

    def update(self, model_shot):
        for var in model_shot.trainable_variables:
            self.weight_copy[var.name] += self.mu * var

    def set_weights(self, avg_model):
        for var in avg_model.trainable_variables:
            var = self.weight_copy[var.name]

@tf.function
def forward_model(data_iter):
    def forward_step_fn(images, labels):
        tmp = model(images, training=True)
        
    # this loop runs on the TPU
    for _ in tf.range(STEPS_PER_EPOCH):
        strategy.experimental_run_v2(forward_step_fn, next(data_iter))

In [25]:
# val_dataset = get_validation_dataset()
# images_ds = val_dataset.map(lambda image, label: image)
# labels_ds = val_dataset.map(lambda image, label: label).unbatch()
# valid_labels = next(iter(labels_ds.batch(NUM_VALIDATION_IMAGES))).numpy()
# valid_labels = np.argmax(valid_labels, -1)
# valid_labels

In [26]:
# %%time

# model.load_weights(MODEL_PATH + 'best_model.h5')
# valid_preds = model.predict(images_ds)
# valid_preds = np.argmax(valid_preds, axis=-1)

In [27]:
# %%time

# # ensemble 4 snapshots from cyclic training

# # model.load_weights(MODEL_PATH + 'best_model.h5')
# # valid_preds = model.predict(images_ds)
# for i in range(4):
#     model.load_weights(MODEL_PATH_2 + f'best_model_{i}.h5')
#     if i==0: valid_preds = model.predict(images_ds)
#     else: valid_preds += model.predict(images_ds)
# #     valid_preds += model.predict(images_ds)

# valid_preds = np.argmax(valid_preds, axis=-1)

In [28]:
# val_acc = accuracy_score(valid_labels, valid_preds)
# val_f1 = f1_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')
# val_prec = precision_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')
# val_rec = recall_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')

# print('accuracy: {:.3f}, f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(val_acc, val_f1, val_prec, val_rec))

## TTA

In [29]:
# %%time

# valid_preds = TTA(8, model, beta=0.3, mode='valid')
# valid_preds = np.argmax(valid_preds, axis=-1)

In [30]:
# %%time

# # ensemble 4 snapshots with TTA from cyclic training

# # model.load_weights(MODEL_PATH + 'best_model.h5')
# # valid_preds = TTA(8, model, beta=0.3, mode='valid')
# for i in range(4):
#     model.load_weights(MODEL_PATH_2 + f'best_model_{i}.h5')
#     if i==0: valid_preds = TTA(8, model, beta=0.3, mode='valid')
#     else: valid_preds += TTA(8, model, beta=0.3, mode='valid')
# #     valid_preds += TTA(8, model, beta=0.3, mode='valid')

# valid_preds = np.argmax(valid_preds, axis=-1)

In [31]:
# val_acc = accuracy_score(valid_labels, valid_preds)
# val_f1 = f1_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')
# val_prec = precision_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')
# val_rec = recall_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')

# print('accuracy: {:.3f}, f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(val_acc, val_f1, val_prec, val_rec))

## SWA

In [32]:
# %%time

# avg_obj = NNAverage(model, 1/4.)  # 1/5.

# # model.load_weights(MODEL_PATH + 'best_model.h5')
# # avg_obj.update(model)
# for i in range(4):
#     model.load_weights(MODEL_PATH_2 + f'best_model_{i}.h5')
#     avg_obj.update(model)

# avg_obj.set_weights(model)
# train_dist_ds = strategy.experimental_distribute_dataset(get_training_dataset())
# train_data_iter = iter(train_dist_ds)
# forward_model(train_data_iter)

# valid_preds = model.predict(images_ds)
# valid_preds = np.argmax(valid_preds, axis=-1)

In [33]:
# val_acc = accuracy_score(valid_labels, valid_preds)
# val_f1 = f1_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')
# val_prec = precision_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')
# val_rec = recall_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')

# print('accuracy: {:.3f}, f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(val_acc, val_f1, val_prec, val_rec))

In [34]:
# %%time

# valid_preds = TTA(8, model, beta=0.3, mode='valid')
# valid_preds = np.argmax(valid_preds, axis=-1)

In [35]:
# val_acc = accuracy_score(valid_labels, valid_preds)
# val_f1 = f1_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')
# val_prec = precision_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')
# val_rec = recall_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')

# print('accuracy: {:.3f}, f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(val_acc, val_f1, val_prec, val_rec))

In [36]:
# MODEL_PATH_1_1 = '../input/sp-effnb4/'
# MODEL_PATH_1_2 = '../input/sp-effnb4-cycs/'
# MODEL_PATH_1_3 = '../input/sp-effnb4-folds/'

# MODEL_PATH_2_1 = '../input/sp-effnb5/'
# MODEL_PATH_2_2 = '../input/sp-effnb5-cycs/'
# MODEL_PATH_2_3 = '../input/sp-effnb5-folds/'
# MODEL_PATH_2_4 = '../input/sp-effnb5-folds-2/'

# Predictions

In [37]:
test_ds = get_test_dataset()
test_images_ds = test_ds.map(lambda image, filename: image)

In [38]:
# %%time

# print('Computing predictions...')
# test_images_ds = test_ds.map(lambda image, filename: image)
# probabilities = model.predict(test_images_ds)
# # probabilities = TTA(8, model, beta=0.3)
# predictions = np.argmax(probabilities, axis=-1)
# predictions = [CLASSES[c] for c in predictions]

In [39]:
# %%time

# print('Computing predictions...')
# # avg_obj = NNAverage(model, 1/4.)  # 1/5.

# # model.load_weights(MODEL_PATH + 'best_model.h5')
# # avg_obj.update(model)
# # probabilities = model.predict(test_images_ds)
# # probabilities = TTA(8, model, beta=0.3)
# for i in range(4):
#     model.load_weights(MODEL_PATH_2 + f'best_model_{i}.h5')
# #     avg_obj.update(model)
#     if i==0:
# #         probabilities = model.predict(test_images_ds)
#         probabilities = TTA(8, model, beta=0.3)
#     else:
# #         probabilities += model.predict(test_images_ds)
#         probabilities += TTA(8, model, beta=0.3)
# #     probabilities += model.predict(test_images_ds)
# #     probabilities += TTA(8, model, beta=0.3)

# # avg_obj.set_weights(model)
# # train_dist_ds = strategy.experimental_distribute_dataset(get_training_dataset())
# # train_data_iter = iter(train_dist_ds)
# # forward_model(train_data_iter)
# # probabilities += TTA(8, model, beta=0.3)

# predictions = np.argmax(probabilities, axis=-1)
# predictions = [CLASSES[c] for c in predictions]

In [40]:
# %%time

# print('Computing predictions...')
# # avg_obj = NNAverage(model, 1/6.)

# model.load_weights(MODEL_PATH + 'best_model.h5')
# # avg_obj.update(model)
# probabilities = model.predict(test_images_ds)
# # probabilities = TTA(8, model, beta=0.3)
# for i in range(5):
#     model.load_weights(MODEL_PATH_3 + f'best_model_{i}.h5')
# #     avg_obj.update(model)
#     probabilities += model.predict(test_images_ds)
# #     probabilities = TTA(8, model, beta=0.3)

# # avg_obj.set_weights(model)
# # train_dist_ds = strategy.experimental_distribute_dataset(get_training_dataset())
# # train_data_iter = iter(train_dist_ds)
# # forward_model(train_data_iter)
# # probabilities += TTA(8, model, beta=0.3)

# predictions = np.argmax(probabilities, axis=-1)
# predictions = [CLASSES[c] for c in predictions]

In [41]:
MODEL_PATH_1_1 = '../input/speffnb4/'
MODEL_PATH_1_2 = '../input/speffnb4cycs/'
MODEL_PATH_1_3 = '../input/speffnb4folds/'

MODEL_PATH_2_1 = '../input/speffnb5/'
MODEL_PATH_2_2 = '../input/speffnb5cycs/'
MODEL_PATH_2_3 = '../input/speffnb5folds/'
MODEL_PATH_2_4 = '../input/speffnb5folds2/'

In [42]:
%%time

print('Computing predictions...')

model_1.load_weights(MODEL_PATH_1_1 + f'best_model.h5')
probabilities_cyc = model_1.predict(test_images_ds)
# probabilities_cyc = TTA(8, model_1, beta=0.3)
for i in range(4):
    model_1.load_weights(MODEL_PATH_1_2 + f'best_model_{i}.h5')
    probabilities_cyc += model_1.predict(test_images_ds)
#     probabilities_cyc += TTA(8, model_1, beta=0.3)

model_2.load_weights(MODEL_PATH_2_1 + f'best_model.h5')
probabilities_cyc += model_2.predict(test_images_ds)
# probabilities_cyc += TTA(8, model_2, beta=0.3)
for i in range(4):
    model_2.load_weights(MODEL_PATH_2_2 + f'best_model_{i}.h5')
    probabilities_cyc += model_2.predict(test_images_ds)
#     probabilities_cyc += TTA(8, model_2, beta=0.3)

predictions = np.argmax(probabilities_cyc, axis=-1)
predictions = [CLASSES[c] for c in predictions]

Computing predictions...
CPU times: user 1min 40s, sys: 31.8 s, total: 2min 12s
Wall time: 3min 9s


In [43]:
%%time

print('Computing predictions...')

for i in range(3):
    model_1.load_weights(MODEL_PATH_1_3 + f'best_model_{i}.h5')
    if i==0:
        probabilities_cv = model_1.predict(test_images_ds)
#         probabilities_cv = TTA(8, model_1, beta=0.3)
    else:
        probabilities_cv += model_1.predict(test_images_ds)
#         probabilities_cv += TTA(8, model_1, beta=0.3)

for i in range(3):
    model_2.load_weights(MODEL_PATH_2_3 + f'best_model_{i}.h5')
    probabilities_cv += model_2.predict(test_images_ds)
#     probabilities_cv += TTA(8, model_2, beta=0.3)

for i in range(3):
    model_2.load_weights(MODEL_PATH_2_4 + f'best_model_{i}.h5')
    probabilities_cv += model_2.predict(test_images_ds)
#     probabilities_cv += TTA(8, model_2, beta=0.3)

predictions = np.argmax(probabilities_cv, axis=-1)
predictions = [CLASSES[c] for c in predictions]

Computing predictions...
CPU times: user 1min 19s, sys: 28.4 s, total: 1min 47s
Wall time: 2min 9s


In [44]:
probabilities = probabilities_cv + probabilities_cyc

predictions = np.argmax(probabilities, axis=-1)
predictions = [CLASSES[c] for c in predictions]

In [45]:
print('Generating submission.csv file...')
test_fns_ds = test_ds.map(lambda image, filename: filename).unbatch()
test_fns = next(iter(test_fns_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
test_fns = [fn.split('/')[-1] for fn in test_fns]

np.savetxt('submission.csv', np.rec.fromarrays([test_fns, predictions]), fmt=['%s', '%s'], delimiter=',', header='filename,category', comments='')
!head submission.csv

Generating submission.csv file...
filename,category
fd663cf2b6e1d7b02938c6aaae0a32d2.jpg,20
3e1b4fcb5f15a2b6eb9b188bec9ac98c.jpg,07
c7fd77508a8c355eaab0d4e10efd6b15.jpg,27
13f2954fa62bde63320bf1bd313964f8.jpg,32
127f3e6d6e3491b2459812353f33a913.jpg,04
6c6b59b94fcea916e1d49996611dabc7.jpg,38
5ca4f2da11eda083064e6c36f37eeb81.jpg,22
efbe090bdd5129de37da14901448566d.jpg,34
46d681a542f2c71be017eef6aae23313.jpg,12
